In [112]:
## Import packages
import telegram
import matplotlib.path as mplPath
import pandas as pd
import os 
import numpy as np
import time

In [116]:
def telegram_push(text):
    chat_id = '@TheMatrixISE534'
    token = '5716566185:AAGr0Eg2HFQ3qAOG_P-42p7-ha1r7dGKRYk'
    bot = telegram.Bot(token = token)
    bot.send_message(chat_id = chat_id, text = text)


def location_exam(lat, lng):
    ### coordinates of buidling corners
    A = (51.46109067323197, -0.9326277978872678)
    B = (51.46102714195754, -0.9320514526904464)
    C = (51.46028409570366, -0.9322243562494928)
    D = (51.460317243006244, -0.9324371606298576)
    E = (51.46023437470467, -0.9325036619987218)
    F = (51.46023989926278, -0.9328583359659965)
    G = (51.46032276755434, -0.9328672028151782)
    H = (51.4603421034674, -0.9334036471906814)
    I = (51.4606625259766, -0.933368179793954)
    J = (51.460645952453696, -0.9326854324069501)

    poly = [A,B,C,D,E,F,G,H,I,J]
    poly_path = mplPath.Path(poly)
    D = (lat, lng)
    return poly_path.contains_points([D])[0]



In [43]:
fixed_data = pd.read_csv("uk_00_fixed_1F.csv")
mobile_data = pd.read_csv("uk_00_mobile_1F.csv")
# time format
fixed_data['format_time'] = fixed_data['localtime'].str.slice(0, -4)
fixed_data['format_time'] = pd.to_datetime(fixed_data['format_time'], format = '%Y-%m-%d %H:%M:%S')

In [133]:
for i in range(len(fixed_data)):
    asset_id = fixed_data.iloc[i, 0]
    login_time = fixed_data.iloc[i, 7]
    lat = fixed_data.iloc[i, 4]
    lng = fixed_data.iloc[i, 5]
    ## 位置判定
    if (not location_exam(lat, lng)):
        telegram_push("WARNING: The asset "+str(asset_id)+" on the "+fixed_data.iloc[i, 2]+" is out of bound!!")
        time.sleep(1)
    ## 登陆时间判定（仅对固定资产？）
    if (login_time.hour >= 2 and login_time.hour < 6):
        telegram_push("WARNING: The asset "+str(asset_id)+" on the "+fixed_data.iloc[i, 2]+" is logging in at irregular time!!")
        time.sleep(2)
## telegram 推送报错 exceeded

In [130]:

count_out = 0
count_time = 0
for i in range(len(fixed_data)):
    lat = fixed_data.iloc[i, 4]
    lng = fixed_data.iloc[i, 5]
    login_time = fixed_data.iloc[i, 7]
    if (not location_exam(lat, lng)):
        count_out += 1
        ## 打印出界的samples
        print((lat, lng))
    elif (login_time.hour >= 2 and login_time.hour < 6):
        count_time += 1
        print(login_time.hour)
count_out/len(fixed_data)
## 百分之四的设备在2-6点间登陆
count_time/len(fixed_data)

4
5
5
5
5
4
4
4
4
4
4
4
4
4


0.004319654427645789

In [127]:
for i in range(len(fixed_data)):
    login_time = fixed_data.iloc[i, 7]
    print(login_time.hour)
    print(login_time.hour >= 2)
    print(login_time.hour < 6)
    print(login_time.hour >= 2 and login_time.hour < 6)

9
True
False
False
4
True
True
True
18
True
False
False
17
True
False
False
13
True
False
False
12
True
False
False
10
True
False
False
8
True
False
False
8
True
False
False
8
True
False
False
8
True
False
False
8
True
False
False
8
True
False
False
8
True
False
False
8
True
False
False
8
True
False
False
8
True
False
False
8
True
False
False
8
True
False
False
8
True
False
False
19
True
False
False
14
True
False
False
11
True
False
False
11
True
False
False
11
True
False
False
11
True
False
False
11
True
False
False
11
True
False
False
11
True
False
False
11
True
False
False
11
True
False
False
11
True
False
False
11
True
False
False
11
True
False
False
11
True
False
False
13
True
False
False
13
True
False
False
13
True
False
False
13
True
False
False
13
True
False
False
13
True
False
False
13
True
False
False
13
True
False
False
13
True
False
False
13
True
False
False
13
True
False
False
13
True
False
False
13
True
False
False
8
True
False
False
5
True
True
True
5
True
True
True
5
Tr

In [129]:
print(True and False)

False


In [ ]:
# 关于对于mobile（移动人员）的功能的实现



## 功能描述

1. 使用 mobile数据表（用一个就可以）， 对mobile数据表加入 “权限列”-----Priority。这一列就用dummy value随机填充就行，就0，1 两个权限。1权限哪都能去，0有些区域不能去。不能去的区域（restricted area）划出来一块儿就行（随便找个对角线画一下就行）
2. 不用遍历整个表列，读取dataframe中最新（时间最近）的10个就可以。对这十个进行逻辑判断：priority = 0 & position in restricted area------------>  telegram 发消息：xxxxxx not allowed to enter restricted area

In [138]:
mobile_data['priority'] = np.random.randint(0, 2, (len(mobile_data),1))
mobile_data

,Unnamed: 0,Site,Level,ClientMacAddr,lat,lng,localtime,priority
0,0,UK Office,1st Floor,9c:8c:6e:46:1c:5e,51.460445,-0.933048,2020-09-01 23:58:49.582 UTC,1
1,2,UK Office,1st Floor,9c:8c:6e:46:1c:5e,51.460445,-0.933048,2020-09-01 23:58:38.112 UTC,1
2,5,UK Office,1st Floor,9c:8c:6e:46:1c:5e,51.460448,-0.933048,2020-09-01 23:58:29.514 UTC,1
3,7,UK Office,1st Floor,9c:8c:6e:46:1c:5e,51.460445,-0.933048,2020-09-01 23:58:18.046 UTC,0
4,9,UK Office,1st Floor,9c:8c:6e:46:1c:5e,51.460445,-0.933048,2020-09-01 23:58:09.522 UTC,1
...,...,...,...,...,...,...,...,...
3730129,3633851,UK Office,1st Floor,a4:6c:f1:41:f3:9c,51.460351,-0.932475,2020-03-24 12:20:44.473 UTC,0
3730130,3641143,UK Office,1st Floor,ec:1f:72:28:28:9d,51.460364,-0.932535,2020-03-24 08:03:46.662 UTC,0
3730131,3677403,UK Office,1st Floor,ec:1f:72:28:28:9d,51.460364,-0.932514,2020-03-23 10:25:49.219 UTC,1
3730132,3677410,UK Office,1st Floor,ec:1f:72:28:28:9d,51.460364,-0.932514,2020-03-23 10:25:35.532 UTC,1


In [137]:
def priority_exam(lat, lng, priority):
    ### coordinates of buidling corners
    A = (51.46109067323197, -0.9326277978872678)
    B = (51.46102714195754, -0.9320514526904464)
    C = (51.46028409570366, -0.9322243562494928)
    D = (51.460317243006244, -0.9324371606298576)
    E = (51.46023437470467, -0.9325036619987218)
    F = (51.46023989926278, -0.9328583359659965)
    G = (51.46032276755434, -0.9328672028151782)
    H = (51.4603421034674, -0.9334036471906814)
    I = (51.4606625259766, -0.933368179793954)
    J = (51.460645952453696, -0.9326854324069501)

    # only priority == 1 can get into this area
    first_poly = [I,J, C, D, E, F, G, H]
    first_poly_path = mplPath.Path(first_poly)
    D = (lat, lng)
    return priority  == 1 or not first_poly_path.contains_points([D])[0] 


In [144]:
for i in range(len(mobile_data)):
    user_id = mobile_data.iloc[i, 0]
    login_time = mobile_data.iloc[i, 6]
    lat = mobile_data.iloc[i, 4]
    lng = mobile_data.iloc[i, 5]
    priority = mobile_data.iloc[i, 7]
    ## mobile位置判定
    if (not priority_exam(lat, lng, priority)):
        telegram_push("WARNING: The user "+str(user_id)+" on the "+fixed_data.iloc[i, 2]+" entered the no premission area!!")
        time.sleep(1)

KeyboardInterrupt: 

In [143]:
mobile_data.iloc[1,7]

1